In [1]:
import os
import random
import matplotlib.pyplot as plt
import cv2

# Diretório do dataset
base_dir = "/kaggle/input/stress-non-stress-images"  # 🔹 Altere para o caminho correto


In [2]:
def get_random_image(folder):
    """Escolhe uma imagem aleatória de uma pasta."""
    folder_path = os.path.join(base_dir, folder)
    
    # Obtém todas as imagens da pasta
    images = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    if images:  # Se houver imagens na pasta
        img_name = random.choice(images)  # Escolhe uma aleatória
        img_path = os.path.join(folder_path, img_name)
        return img_path, img_name
    else:
        return None, None  # Se a pasta estiver vazia


In [3]:
train_dir = "/kaggle/input/stress-non-stress-images/KDEF/KDEF/Train"
test_dir = "/kaggle/input/stress-non-stress-images/KDEF/KDEF/Test"
val_dir = "/kaggle/input/stress-non-stress-images/FINAL_TFEID"

# Função para carregar imagens
def get_images_from_folder(folder, labels=None):
    images = {label: [] for label in labels}  # Criar dicionário para labels fornecidos
    for label in labels:
        label_path = os.path.join(folder, label)
        if os.path.isdir(label_path):
            for img_name in os.listdir(label_path):
                if img_name.lower().endswith((".png", ".jpg", ".jpeg")):
                    images[label].append(os.path.join(label_path, img_name))
    return images

# Carregar Train, Test e Validação
train_images = get_images_from_folder(train_dir, ["Stress", "NoStress"])
test_images = get_images_from_folder(test_dir, ["Stress", "NoStress"])
val_images = get_images_from_folder(val_dir, ["FINALTFEID_STRESS", "FINALTFEID_NONSTRESS"])

# Renomear labels da validação para manter o padrão
val_images["Stress"] = val_images.pop("FINALTFEID_STRESS")
val_images["NoStress"] = val_images.pop("FINALTFEID_NONSTRESS")

# Mostrar contagem de imagens
print(f"Train: {len(train_images['Stress'])} Stress, {len(train_images['NoStress'])} NoStress")
print(f"Validation: {len(val_images['Stress'])} Stress, {len(val_images['NoStress'])} NoStress")
print(f"Test: {len(test_images['Stress'])} Stress, {len(test_images['NoStress'])} NoStress")

# Exibir imagens aleatórias de cada conjunto
show_random_images(train_images, "Train")
show_random_images(val_images, "Validation")
show_random_images(test_images, "Test")

Train: 2449 Stress, 1960 NoStress
Validation: 235 Stress, 120 NoStress
Test: 1049 Stress, 842 NoStress


NameError: name 'show_random_images' is not defined

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)

train_dataset = datagen.flow_from_directory(
    "/kaggle/input/stress-non-stress-images/KDEF/KDEF/Train/",
    target_size=(210, 158),
    batch_size=32,
    class_mode="binary"
)


Found 4409 images belonging to 2 classes.


In [5]:
test_dataset = datagen.flow_from_directory(
    "/kaggle/input/stress-non-stress-images/KDEF/KDEF/Test",
    target_size=(210, 158),
    batch_size=32,
    class_mode="binary"
)


Found 1891 images belonging to 2 classes.


In [6]:
val_dataset = datagen.flow_from_directory(
    "/kaggle/input/stress-non-stress-images/FINAL_TFEID",
    target_size=(210, 158),
    batch_size=32,
    class_mode="binary"
)

Found 355 images belonging to 2 classes.


In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(210, 158, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    # Segunda camada convolucional
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    # Terceira camada convolucional
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2,2)),

    # Flatten para transformar em vetor
    tf.keras.layers.Flatten(),

    # Camada densa totalmente conectada
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Evita overfitting

    # Camada de saída com 2 neurônios (Stress e NoStress)
    tf.keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [10]:
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


138/138 ━━━━━━━━━━━━━━━━━━━━ 78s 498ms/step - accuracy: 0.5156 - loss: 19.5069 - val_accuracy: 0.6592 - val_loss: 0.6711
Epoch 2/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 44s 301ms/step - accuracy: 0.5503 - loss: 0.6876 - val_accuracy: 0.6620 - val_loss: 0.6589
Epoch 3/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 44s 306ms/step - accuracy: 0.5677 - loss: 0.6843 - val_accuracy: 0.6563 - val_loss: 0.6587
Epoch 4/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 45s 308ms/step - accuracy: 0.5754 - loss: 0.6844 - val_accuracy: 0.6366 - val_loss: 0.6694
Epoch 5/5
138/138 ━━━━━━━━━━━━━━━━━━━━ 44s 305ms/step - accuracy: 0.5587 - loss: 0.6838 - val_accuracy: 0.6620 - val_loss: 0.6594


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import EfficientNetB0

# Carregar o modelo base pré-treinado sem a camada de saída (include_top=False)
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(210, 158, 3)) 

# Congelar o modelo base para preservar os pesos pré-treinados
base_model.trainable = False

# Construindo a nova "cabeça" do modelo
x = base_model.output  # Obtém a saída da última camada do modelo base
x = GlobalAveragePooling2D()(x)  # Camada de pooling para reduzir a dimensionalidade
x = Dense(512, activation="relu")(x)  # Camada densa
x = Dropout(0.3)(x)  # Dropout para evitar overfitting
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
output_layer = Dense(1, activation="sigmoid", name="output_layer")(x)  # Saída binária

# Criar o modelo final corretamente
model = Model(inputs=base_model.input, outputs=output_layer)

# Compilar o modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss="binary_crossentropy",  # Para classificação binária (0 = NoStress, 1 = Stress)
              metrics=["accuracy"])

# Resumo do modelo
model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 210, 158, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 210, 158, 3)    │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 210, 158, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 210, 158, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 211, 159, 3)    │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 105, 79, 32)    │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 105, 79, 32)    │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 105, 79, 32)    │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 105, 79, 32)    │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 105, 79, 32)    │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 105, 79, 32)    │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 4,771,236 (18.20 MB)

 Trainable params: 721,665 (2.75 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [12]:
history = model.fit(
    train_dataset,              
    validation_data=val_dataset, 
    epochs=10,                   
    batch_size=32
)

Epoch 1/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 84s 450ms/step - accuracy: 0.5827 - loss: 0.6759 - val_accuracy: 0.5521 - val_loss: 0.6905
Epoch 2/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 45s 311ms/step - accuracy: 0.6742 - loss: 0.6084 - val_accuracy: 0.7155 - val_loss: 0.5968
Epoch 3/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 46s 315ms/step - accuracy: 0.6932 - loss: 0.5678 - val_accuracy: 0.6930 - val_loss: 0.5830
Epoch 4/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 44s 306ms/step - accuracy: 0.7441 - loss: 0.5321 - val_accuracy: 0.6986 - val_loss: 0.5756
Epoch 5/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 45s 309ms/step - accuracy: 0.7544 - loss: 0.5114 - val_accuracy: 0.7127 - val_loss: 0.5639
Epoch 6/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 45s 310ms/step - accuracy: 0.7558 - loss: 0.5040 - val_accuracy: 0.7127 - val_loss: 0.5735
Epoch 7/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 45s 314ms/step - accuracy: 0.7477 - loss: 0.4990 - val_accuracy: 0.7099 - val_loss: 0.5685
Epoch 8/10
138/138 ━━━━━━━━━━━━━━━━━━━━ 45s 313ms/step - accuracy: 0.7641 - loss: 0